<a href="https://colab.research.google.com/github/xR3M/Mexico_City_InSAR_DL/blob/main/Mexico_City_Interferometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import platform, sys, os

In [2]:
if 'google.colab' in sys.modules:
    count = !ls /usr/local | grep GMTSAR | wc -l
    if count == ['0']:
        !export DEBIAN_FRONTEND=noninteractive
        !apt-get update > /dev/null
        !apt install -y csh autoconf gfortran \
            libtiff5-dev libhdf5-dev liblapack-dev libgmt-dev gmt-dcw gmt-gshhg gmt  > /dev/null
        # GMTSAR codes are not so good to be compiled by modern GCC
        !apt install gcc-9 > /dev/null
        !update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 10
        !update-alternatives --config gcc
        !gcc --version | head -n 1
        !rm -fr /usr/local/GMTSAR
        !git config --global advice.detachedHead false
        !cd /usr/local && git clone -q --branch master https://github.com/gmtsar/gmtsar GMTSAR
        # revert recent broken commit
        !cd /usr/local/GMTSAR && git checkout e98ebc0f4164939a4780b1534bac186924d7c998 > /dev/null
        !cd /usr/local/GMTSAR && autoconf > /dev/null
        !cd /usr/local/GMTSAR && ./configure --with-orbits-dir=/tmp > /dev/null
        !cd /usr/local/GMTSAR && make 1>/dev/null 2>/dev/null
        !cd /usr/local/GMTSAR && make install >/dev/null
        # fix for missed script, use bash instead of csh interpretator
        # note: csh messes stdout and stderr in Docker environment, it's resolved in PyGMTSAR code
        !echo '#!/bin/sh' > /usr/local/GMTSAR/bin/gmtsar_sharedir.csh
        !echo echo /usr/local/GMTSAR/share/gmtsar >> /usr/local/GMTSAR/bin/gmtsar_sharedir.csh
        !chmod a+x /usr/local/GMTSAR/bin/gmtsar_sharedir.csh
        !/usr/local/GMTSAR/bin/gmtsar_sharedir.csh
        # test one GMTSAR binary
        !/usr/local/GMTSAR/bin/make_s1a_tops 2>&1 | head -n 2





update-alternatives: using /usr/bin/gcc-9 to provide /usr/bin/gcc (gcc) in auto mode
There is only one alternative in link group gcc (providing /usr/bin/gcc): /usr/bin/gcc-9
Nothing to configure.
gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
HEAD is now at e98ebc0 extending solid_tide to 2050 as leap seconds are abandoned hopefully
/usr/local/GMTSAR/share/gmtsar
 
Usage: make_slc_s1a_tops xml_file tiff_file output mode dr.grd da.grd


In [3]:
import sys
if 'google.colab' in sys.modules:
    !apt install -y xvfb > /dev/null
    !{sys.executable} -m pip install pyvista xvfbwrapper > /dev/null
    import xvfbwrapper
    display = xvfbwrapper.Xvfb(width=800, height=600)
    display.start()

In [ ]:
# use default GMTSAR installation path
PATH = os.environ['PATH']
if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + ':/usr/local/GMTSAR/bin/'
    %env PATH {PATH}

In [5]:
!{sys.executable} --version

Python 3.10.12


In [6]:
if 'google.colab' in sys.modules:
    #!{sys.executable} -m pip install -qU git+https://github.com/mobigroup/gmtsar.git@pygmtsar2#subdirectory=pygmtsar
    !{sys.executable} -m pip install -q pygmtsar
import pygmtsar
pygmtsar.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:

'2024.6.3'

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from dask.distributed import Client
import dask
import warnings
warnings.filterwarnings('ignore')
# plotting modules
import pyvista as pv
# magic trick for white background
pv.set_plot_theme("document")
import panel
panel.extension('vtk')
from contextlib import contextmanager
import matplotlib.pyplot as plt
@contextmanager
def mpl_settings(settings):
    original_settings = {k: plt.rcParams[k] for k in settings}
    plt.rcParams.update(settings)
    yield
    plt.rcParams.update(original_settings)
plt.rcParams['figure.figsize'] = [12, 4]
plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.titlesize'] = 24
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
%matplotlib inline

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

In [34]:
SCENES = """
S1A_IW_SLC__1SDV_20240421T004848_20240421T004915_053525_067F0A_A009
S1A_IW_SLC__1SDV_20240409T004848_20240409T004915_053350_067822_B3C1
S1A_IW_SLC__1SDV_20240328T004847_20240328T004914_053175_067152_C9AA
S1A_IW_SLC__1SDV_20240316T004847_20240316T004914_053000_066A9D_7836
S1A_IW_SLC__1SDV_20240304T004847_20240304T004914_052825_066491_C450
S1A_IW_SLC__1SDV_20240221T004847_20240221T004914_052650_065EA1_F5CD
S1A_IW_SLC__1SDV_20240209T004847_20240209T004914_052475_0658AE_9906
S1A_IW_SLC__1SDV_20240128T004847_20240128T004914_052300_0652C5_08A5
S1A_IW_SLC__1SDV_20240116T004847_20240116T004914_052125_064CE3_0CA5
S1A_IW_SLC__1SDV_20240104T004848_20240104T004915_051950_0646E9_491C
S1A_IW_SLC__1SDV_20231223T004849_20231223T004916_051775_0640EA_3264
S1A_IW_SLC__1SDV_20231211T004850_20231211T004917_051600_063AD7_A98B
S1A_IW_SLC__1SDV_20231129T004850_20231129T004917_051425_0634C5_65C0
S1A_IW_SLC__1SDV_20231117T004851_20231117T004917_051250_062EBA_EFAE
S1A_IW_SLC__1SDV_20231105T004851_20231105T004918_051075_0628AB_8A54
S1A_IW_SLC__1SDV_20231024T004851_20231024T004918_050900_0622AE_289A
S1A_IW_SLC__1SDV_20231012T004850_20231012T004917_050725_061CBD_AB0E
S1A_IW_SLC__1SDV_20230930T004851_20230930T004918_050550_0616B6_8CC6
S1A_IW_SLC__1SDV_20230918T004851_20230918T004918_050375_0610BA_BCA1
S1A_IW_SLC__1SDV_20230906T004850_20230906T004917_050200_060ACB_D4DD
S1A_IW_SLC__1SDV_20230825T004850_20230825T004917_050025_0604C7_0C53
S1A_IW_SLC__1SDV_20230813T004849_20230813T004916_049850_05FED5_9DA6
S1A_IW_SLC__1SDV_20230801T004848_20230801T004915_049675_05F92D_359D
S1A_IW_SLC__1SDV_20230720T004844_20230720T004916_049500_05F3C5_F156
S1A_IW_SLC__1SDV_20230708T004847_20230708T004914_049325_05EE6E_CDB3
S1A_IW_SLC__1SDV_20230626T004846_20230626T004913_049150_05E906_F71B
S1A_IW_SLC__1SDV_20230614T004845_20230614T004912_048975_05E3BC_2DAF
S1A_IW_SLC__1SDV_20230602T004845_20230602T004912_048800_05DE62_1ABF
S1A_IW_SLC__1SDV_20230521T004844_20230521T004911_048625_05D92B_BBE6
S1A_IW_SLC__1SDV_20230509T004844_20230509T004911_048450_05D402_1A7E
S1A_IW_SLC__1SDV_20230427T004843_20230427T004910_048275_05CE23_ED5F
S1A_IW_SLC__1SDV_20230415T004843_20230415T004910_048100_05C844_2F06
S1A_IW_SLC__1SDV_20230403T004842_20230403T004909_047925_05C255_801F
S1A_IW_SLC__1SDV_20230322T004842_20230322T004909_047750_05BC7E_E426
S1A_IW_SLC__1SDV_20230310T004842_20230310T004909_047575_05B68E_E04C
S1A_IW_SLC__1SDV_20230226T004842_20230226T004909_047400_05B0A6_6949
S1A_IW_SLC__1SDV_20230214T004842_20230214T004909_047225_05AAAA_B9C7
S1A_IW_SLC__1SDV_20230202T004842_20230202T004909_047050_05A4CF_6AC3
S1A_IW_SLC__1SDV_20230121T004842_20230121T004909_046875_059EF3_B488
S1A_IW_SLC__1SDV_20230109T004843_20230109T004910_046700_05990A_6B71
S1A_IW_SLC__1SDV_20221228T004844_20221228T004911_046525_05932B_C9F7
S1A_IW_SLC__1SDV_20221216T004844_20221216T004911_046350_058D32_579C
S1A_IW_SLC__1SDV_20221204T004845_20221204T004912_046175_05873C_F90B
S1A_IW_SLC__1SDV_20221122T004846_20221122T004913_046000_058147_FCF5
S1A_IW_SLC__1SDV_20221110T004845_20221110T004912_045825_057B63_CFAE
S1A_IW_SLC__1SDV_20221029T004846_20221029T004913_045650_057573_553D
S1A_IW_SLC__1SDV_20221005T004846_20221005T004913_045300_056A69_8BC2
S1A_IW_SLC__1SDV_20220923T004845_20220923T004912_045125_056481_F3D4
S1A_IW_SLC__1SDV_20220911T004846_20220911T004913_044950_055E9E_F4FA
S1A_IW_SLC__1SDV_20220830T004845_20220830T004912_044775_0558C3_6300
S1A_IW_SLC__1SDV_20220806T004844_20220806T004911_044425_054D2C_E8C1
S1A_IW_SLC__1SDV_20220725T004843_20220725T004910_044250_05480B_8EDB
S1A_IW_SLC__1SDV_20220713T004842_20220713T004909_044075_0542D5_60CE
S1A_IW_SLC__1SDV_20220701T004842_20220701T004908_043900_053D9E_CC01
S1A_IW_SLC__1SDV_20220619T004841_20220619T004908_043725_053861_F1AC
S1A_IW_SLC__1SDV_20220607T004840_20220607T004907_043550_05332F_D82E
S1A_IW_SLC__1SDV_20220526T004839_20220526T004906_043375_052DFE_E545
S1A_IW_SLC__1SDV_20220514T004838_20220514T004905_043200_0528CA_D5E8
S1A_IW_SLC__1SDV_20220502T004837_20220502T004904_043025_052320_303C
S1A_IW_SLC__1SDV_20220420T004837_20220420T004904_042850_051D58_A2BC
S1A_IW_SLC__1SDV_20220408T004836_20220408T004903_042675_051776_0479
S1A_IW_SLC__1SDV_20220327T004836_20220327T004903_042500_051193_277B
S1A_IW_SLC__1SDV_20220315T004836_20220315T004903_042325_050B9F_6CA0
S1A_IW_SLC__1SDV_20220303T004836_20220303T004903_042150_0505B4_ED86
S1A_IW_SLC__1SDV_20220219T004836_20220219T004903_041975_04FFB2_04C7
S1A_IW_SLC__1SDV_20220207T004836_20220207T004903_041800_04F99F_C315
S1A_IW_SLC__1SDV_20220126T004837_20220126T004904_041625_04F39C_CB3A
S1A_IW_SLC__1SDV_20220114T004837_20220114T004904_041450_04EDA1_EEE9
S1A_IW_SLC__1SDV_20220102T004838_20220102T004905_041275_04E7DD_21EC
S1A_IW_SLC__1SDV_20211221T004838_20211221T004905_041100_04E1FF_7796
S1A_IW_SLC__1SDV_20211209T004839_20211209T004906_040925_04DC27_0A12
S1A_IW_SLC__1SDV_20211127T004839_20211127T004906_040750_04D60E_9C8D
S1A_IW_SLC__1SDV_20211115T004840_20211115T004907_040575_04D00D_0CBA
S1A_IW_SLC__1SDV_20211103T004840_20211103T004907_040400_04C9F0_4C08
S1A_IW_SLC__1SDV_20211022T004840_20211022T004907_040225_04C3D9_765F
S1A_IW_SLC__1SDV_20211010T004840_20211010T004907_040050_04BDBF_F498
S1A_IW_SLC__1SDV_20210928T004840_20210928T004907_039875_04B7B7_EC4F
S1A_IW_SLC__1SDV_20210916T004839_20210916T004906_039700_04B1B8_AF6C
S1A_IW_SLC__1SDV_20210904T004839_20210904T004906_039525_04ABBF_0341
S1A_IW_SLC__1SDV_20210823T004838_20210823T004905_039350_04A5B8_7F06
S1A_IW_SLC__1SDV_20210730T004837_20210730T004904_039000_0499FE_DFFC
S1A_IW_SLC__1SDV_20210718T004836_20210718T004903_038825_0494CB_4D8D
S1A_IW_SLC__1SDV_20210706T004836_20210706T004903_038650_048F8D_4242
S1A_IW_SLC__1SDV_20210624T004835_20210624T004902_038475_048A45_9F82
S1A_IW_SLC__1SDV_20210612T004834_20210612T004901_038300_04850E_91FF
S1A_IW_SLC__1SDV_20210531T004834_20210531T004900_038125_047FDE_FCE2
S1A_IW_SLC__1SDV_20210519T004833_20210519T004900_037950_047A9D_E8F1
S1A_IW_SLC__1SDV_20210507T004832_20210507T004859_037775_047555_75C5
S1A_IW_SLC__1SDV_20210425T004832_20210425T004859_037600_046F51_41DC
S1A_IW_SLC__1SDV_20210413T004831_20210413T004858_037425_046947_A97D
S1A_IW_SLC__1SDV_20210401T004831_20210401T004858_037250_04633D_EE19
S1A_IW_SLC__1SDV_20210320T004830_20210320T004857_037075_045D33_C4C3
S1A_IW_SLC__1SDV_20210308T004830_20210308T004857_036900_045718_FB8A
S1A_IW_SLC__1SDV_20210224T004830_20210224T004857_036725_0450FC_D128
S1A_IW_SLC__1SDV_20210212T004831_20210212T004858_036550_044AE4_13B8
S1A_IW_SLC__1SDV_20210131T004831_20210131T004858_036375_0444D2_D89D
S1A_IW_SLC__1SDV_20210119T004831_20210119T004858_036200_043EBF_7BD0
S1A_IW_SLC__1SDV_20210107T004832_20210107T004859_036025_043899_294E
S1A_IW_SLC__1SDV_20210107T004832_20210107T004859_036025_043899_294E
S1A_IW_SLC__1SDV_20201214T004833_20201214T004900_035675_042C79_5D66
S1A_IW_SLC__1SDV_20201202T004833_20201202T004900_035500_042679_99E0
S1A_IW_SLC__1SDV_20201120T004834_20201120T004901_035325_042066_F182
S1A_IW_SLC__1SDV_20201108T004834_20201108T004901_035150_041A4C_59CC
S1A_IW_SLC__1SDV_20201027T004834_20201027T004901_034975_041443_E6D8
S1A_IW_SLC__1SDV_20201015T004834_20201015T004901_034800_040E3E_59DE
S1A_IW_SLC__1SDV_20201003T004834_20201003T004901_034625_040826_ED7D
S1A_IW_SLC__1SDV_20200921T004834_20200921T004901_034450_040203_D4A8
S1A_IW_SLC__1SDV_20200909T004833_20200909T004900_034275_03FBC3_C92E
S1A_IW_SLC__1SDV_20200828T004833_20200828T004900_034100_03F59C_B653
S1A_IW_SLC__1SDV_20200816T004832_20200816T004859_033925_03EF73_E2E6
S1A_IW_SLC__1SDV_20200804T004832_20200804T004859_033750_03E969_E401
S1A_IW_SLC__1SDV_20200723T004831_20200723T004858_033575_03E415_88E2
S1A_IW_SLC__1SDV_20200711T004830_20200711T004857_033400_03DEB2_509E
S1A_IW_SLC__1SDV_20200629T004829_20200629T004856_033225_03D963_ECB1
S1A_IW_SLC__1SDV_20200617T004829_20200617T004856_033050_03D413_5842
S1A_IW_SLC__1SDV_20200605T004828_20200605T004855_032875_03CED0_754D
S1A_IW_SLC__1SDV_20200524T004827_20200524T004854_032700_03C99E_E00D
S1A_IW_SLC__1SDV_20200512T004827_20200512T004853_032525_03C451_B4F8
S1A_IW_SLC__1SDV_20200430T004826_20200430T004853_032350_03BE8B_3BE0
S1A_IW_SLC__1SDV_20200418T004825_20200418T004852_032175_03B867_F1A4
S1A_IW_SLC__1SDV_20200406T004825_20200406T004852_032000_03B23A_0951
S1A_IW_SLC__1SDV_20200325T004825_20200325T004852_031825_03AC19_0EB6
S1A_IW_SLC__1SDV_20200313T004825_20200313T004851_031650_03A5E6_DB33
S1A_IW_SLC__1SDV_20200301T004824_20200301T004851_031475_039FD3_57E2
S1A_IW_SLC__1SDV_20200218T004825_20200218T004851_031300_0399D4_AA53
S1A_IW_SLC__1SDV_20200125T004825_20200125T004852_030950_038DA9_947E
S1A_IW_SLC__1SDV_20200113T004825_20200113T004852_030775_03877F_1609
S1A_IW_SLC__1SDV_20200101T004826_20200101T004853_030600_03816B_2704
S1A_IW_SLC__1SDV_20191220T004827_20191220T004853_030425_037B60_6983
S1A_IW_SLC__1SDV_20191208T004827_20191208T004854_030250_037551_DA00
S1A_IW_SLC__1SDV_20191126T004827_20191126T004854_030075_036F51_D2B6
S1A_IW_SLC__1SDV_20191114T004828_20191114T004855_029900_036940_090D
S1A_IW_SLC__1SDV_20191102T004828_20191102T004855_029725_03631A_48BE
S1A_IW_SLC__1SDV_20191021T004828_20191021T004855_029550_035D07_57D8
S1A_IW_SLC__1SDV_20191009T004828_20191009T004855_029375_0356FE_CF44
S1A_IW_SLC__1SDV_20190927T004828_20190927T004855_029200_0350EE_DF9A
S1A_IW_SLC__1SDV_20190915T004827_20190915T004854_029025_034AFD_E759
S1A_IW_SLC__1SDV_20190903T004827_20190903T004854_028850_0344E8_CCAE
S1A_IW_SLC__1SDV_20190822T004826_20190822T004853_028675_033ED2_0E8A
S1A_IW_SLC__1SDV_20190810T004826_20190810T004852_028500_0338BB_11CA
S1A_IW_SLC__1SDV_20190729T004825_20190729T004852_028325_03334B_4F79
S1A_IW_SLC__1SDV_20190717T004824_20190717T004851_028150_032DF4_5B17
S1A_IW_SLC__1SDV_20190705T004823_20190705T004850_027975_0328A5_6D67
S1A_IW_SLC__1SDV_20190623T004822_20190623T004849_027800_03235E_B5B7
S1A_IW_SLC__1SDV_20190611T004821_20190611T004848_027625_031E21_3650
S1A_IW_SLC__1SDV_20190530T004821_20190530T004848_027450_0318C7_6BD5
S1A_IW_SLC__1SDV_20190518T004820_20190518T004847_027275_031350_2ACD
S1A_IW_SLC__1SDV_20190506T004820_20190506T004847_027100_030DD1_C80E
S1A_IW_SLC__1SDV_20190424T004819_20190424T004846_026925_03076D_A658
S1A_IW_SLC__1SDV_20190412T004819_20190412T004846_026750_030118_FE00
S1A_IW_SLC__1SDV_20190331T004819_20190331T004846_026575_02FAB1_A40E
S1A_IW_SLC__1SDV_20190319T004818_20190319T004845_026400_02F442_8D82
S1A_IW_SLC__1SDV_20190307T004818_20190307T004845_026225_02EDC8_BD19
S1A_IW_SLC__1SDV_20190223T004818_20190223T004845_026050_02E789_58A5
S1A_IW_SLC__1SDV_20190211T004818_20190211T004845_025875_02E153_DA11
S1A_IW_SLC__1SDV_20190118T004819_20190118T004846_025525_02D49F_B940
S1A_IW_SLC__1SDV_20190106T004819_20190106T004846_025350_02CE4F_4D57
"""
SCENES = list(filter(None, SCENES.split('\n')))
print (f'Scenes defined: {len(SCENES)}')

ORBIT     = 'A'
SUBSWATH  = 3
REFERENCE = '2022-01-02'

Scenes defined: 157


In [17]:
from pygmtsar import S1, Stack, tqdm_dask, ASF, Tiles, XYZTiles

In [18]:
WORKDIR = 'raw_mex'
DATADIR = 'data_mex'
# define DEM filename outside data directory
DEM = 'dem.nc'

In [19]:
# Define the GeoJSON string for the Polygon
geojson_polygon = '''
{
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [-99.009015, 19.294078],
        [-99.256117, 19.294078],
        [-99.256117, 19.520101],
        [-99.009015, 19.520101],
        [-99.009015, 19.294078]
      ]
    ]
  }
}
'''

# Convert the GeoJSON string to a GeoDataFrame
AOI = gpd.GeoDataFrame.from_features([json.loads(geojson_polygon)])

# Print the GeoDataFrame to verify
print(AOI)

                                                                                              geometry
0  POLYGON ((-99.00902 19.29408, -99.25612 19.29408, -99.25612 19.52010, -99.00902 19.52010, -99.00...


In [20]:
# define the buffer to cover only one burst
BUFFER = 0.035
AOI['geometry'] = AOI.buffer(BUFFER)

In [21]:
# Define the GeoJSON string for the point in Mexico City
geojson_mexico_city = '''
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [-99.13356332073042, 19.43393365718681]
  },
  "properties": {}
}
'''

# Convert the GeoJSON string to a GeoDataFrame
POI= gpd.GeoDataFrame.from_features([json.loads(geojson_mexico_city)])

# Print the GeoDataFrame to verify
print(POI)

                     geometry
0  POINT (-99.13356 19.43393)


In [22]:
# Set these variables to None and you will be prompted to enter your username and password below.
asf_username = 'GoogleColab2023'
asf_password = 'GoogleColab_2023'

In [36]:
# Set these variables to None and you will be prompted to enter your username and password below.
asf = ASF(asf_username, asf_password)
# Optimized scene downloading from ASF - only the required subswaths and polarizations.
print(asf.download_scenes(DATADIR, SCENES, SUBSWATH))

ASF Downloading Sentinel-1 SLC::   0%|          | 0/16 [00:00<?, ?it/s]

OSError: [Errno 28] No space left on device

In [ ]:
# scan the data directory for SLC scenes and download missed orbits
S1.download_orbits(DATADIR, S1.scan_slc(DATADIR))

In [ ]:
# download SRTM Global DEM 1 arc-second
Tiles().download_dem_srtm(AOI, filename=DEM).plot.imshow()
AOI.boundary.plot(ax=plt.gca(), color='red')

In [ ]:
# simple Dask initialization
if 'client' in globals():
    client.close()
client = Client()
client

In [ ]:
scenes = S1.scan_slc(DATADIR, subswath=SUBSWATH)
sbas = Stack(WORKDIR, drop_if_exists=True).set_scenes(scenes).set_reference(REFERENCE)
sbas.to_dataframe()

In [ ]:
sbas.plot_scenes(AOI=AOI)

In [ ]:
sbas.compute_reframe(AOI)
sbas.plot_scenes(AOI=AOI)

In [ ]:
# define the area of interest (AOI) to speedup the processing
sbas.load_dem(DEM, AOI)
sbas.plot_scenes(AOI=AOI)

In [ ]:
sbas.compute_align()

In [ ]:
# bursts-cropped GeoTIFF files moved into backup directory
sbas.backup('backup')
# free disk space removing the cropped Sentinel-1 GeoTIFFs
# alternatively, drop the data directory and use the backup
!rm -fr backup